In [26]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [27]:
os.chdir('/Users/colerichmond/Documents/GitHub/DSC180B-Capstone')
!pwd

/Users/colerichmond/Documents/GitHub/DSC180B-Capstone


In [28]:
!python run.py data

Ingesting data...
...done!


In [29]:
!pwd

/Users/colerichmond/Documents/GitHub/DSC180B-Capstone


In [54]:
x = pd.read_csv('data/raw/sc_stops.csv')
x['officer_race'].value_counts()

white                     445
black                      45
hispanic                    6
other                       2
asian/pacific islander      2
Name: officer_race, dtype: int64

In [40]:
s = "wa_stops.csv"
s.split('_')[0].upper()

'WA'

In [55]:
def get_clean_stops(pth, df):
        
    state = pth.split('_')[0].upper()

    df = clean_races(df)
    df["county_name"] = df["county_name"].apply(clean_county, args=[state])

    return df

def clean_races(df):

    race_map = {"white" : "White",
                "black" : "Black",
                "hispanic" : "Hispanic",
                "asian/pacific islander" : "Asian"}

    df["subject_race"] = df["subject_race"].map(race_map)
    df["officer_race"] = df["officer_race"].map(race_map)
    
    return df

def clean_county(county, state):

    county = county + ", " + state
    
    return county

In [56]:
get_clean_stops('sc_stops.csv', x)

,Unnamed: 0,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,officer_race,type,violation,arrest_made,citation_issued,outcome,search_conducted
0,0,1,2006-02-12,US Prmary 17,"Georgetown County, SC",33,White,male,White,vehicular,56-10-225 Other Violations,False,True,citation,False
1,1,2,2007-10-17,SC Primary 86,"Anderson County, SC",17,White,male,White,vehicular,NaN,False,False,NaN,False
2,2,3,2007-03-20,SC Primary 11,"Greenville County, SC",68,White,male,White,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,False
3,3,4,2007-07-02,US Prmary 29,"Spartanburg County, SC",27,Black,female,White,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,False
4,4,5,2008-11-08,Interstate 20,"Darlington County, SC",29,White,male,Black,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,496,2016-01-21,Secondary 167,"Greenville County, SC",51,White,female,White,vehicular,56-5-2350 No Right of Way,False,True,citation,False
496,496,497,2009-12-19,US Prmary 123,"Oconee County, SC",33,White,male,White,vehicular,NaN,False,False,NaN,False
497,497,498,2009-12-26,Interstate 26,"Newberry County, SC",21,White,male,White,vehicular,56-5-1520 Speeding or Too Fast For Conditions ...,False,True,citation,False
498,498,499,2010-11-08,Interstate 85,"Spartanburg County, SC",19,White,male,White,vehicular,56-5-6520 Other Violations,False,True,citation,False
